In [ ]:
import requests
import PySide2.QtWidgets as QtWidgets
import PySide2.QtGui as QtGui
import PySide2.QtCore as QtCore

# Define the API endpoint and default query parameters
api_url = 'https://api.cpicpgx.org/v1'
default_drug = 'Warfarin'
default_gene = 'CYP2C9'

# Retrieve the list of drugs from the API
drug_list = requests.get(f'{api_url}/drugs').json()
drug_options = [{'label': drug['name'], 'value': drug['name']} for drug in drug_list]


# Retrieve the list of genes from the API
gene_list = requests.get(f'{api_url}/genes').json()
gene_options = [{'label': gene['symbol'], 'value': gene['symbol']} for gene in gene_list]

# Define the app
app = QtWidgets.QApplication([])

# Define the app layout
layout = QtWidgets.QVBoxLayout()

drug_dropdown = QtWidgets.QComboBox()
for option in drug_options:
    drug_dropdown.addItem(option['label'])
drug_dropdown.setCurrentText(default_drug)
layout.addWidget(drug_dropdown)

gene_dropdown = QtWidgets.QComboBox()
for option in gene_options:
    gene_dropdown.addItem(option['label'])
gene_dropdown.setCurrentText(default_gene)
layout.addWidget(gene_dropdown)

output_label = QtWidgets.QLabel()
layout.addWidget(output_label)

widget = QtWidgets.QWidget()
widget.setLayout(layout)
widget.show()

# Define the app callback
def get_recommendation(drug_name, gene_symbol):
    # Retrieve the CPIC recommendation for the selected drug-gene pair
    url = f'{api_url}/guidelines/search?drug={drug_name}&gene={gene_symbol}'
    response = requests.get(url)
    response_json = response.json()
    recommendations = [r['recommendation'] for r in response_json['results']]

    # Update the output label with the CPIC recommendation
    if recommendations:
        output_label.setText('\n\n'.join(recommendations))
    else:
        output_label.setText(f'No CPIC recommendation found for {drug_name} and {gene_symbol}')

# Connect the app callback to the dropdowns
drug_dropdown.currentTextChanged.connect(lambda text: get_recommendation(text, gene_dropdown.currentText()))
gene_dropdown.currentTextChanged.connect(lambda text: get_recommendation(drug_dropdown.currentText(), text))

# Run the app
app.exec_()